# <center>KNN classification avec *scikit-learn*: cas 2 </center>

## Classification multi-classe

Nous avons jusqu'à maintenant utilisé l'algorithme des *k* plus proches voisins pour réaliser de la classification binaire (2 classes). Nous allons maintenant l'utiliser pour réaliser de la classification multi-classe. Nous travaillerons avec le jeu de données *iris*. 
Les données comprennent 150 individus et sont séparées en 2 ensembles. 
- dans le premier ensemble, un individu est représenté par un t_uple qui correspond à la longueur du sépale, la largeur du sépale, la longueur du pétale et à la largeur du pétale. 
- le second ensemble est complémentaire du premier. Il stocke dans chaque ligne, le type d'iris correspondant au t_uple du premier. Les types d'iris sont répartis en 3 classes: *Setosa*, *Versicolor* ou *Virginica*. 

Ce jeu de données est très souvent utilisé pour tester des algorithmes de classification. Le problème à résoudre est de prédire le second ensemble à l'aide du premier.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# chargement du dataset de fleur d'iris
iris = load_iris()
X = iris.data # X contient 150 échantillons, 4 variables caractéristiques
Y = iris.target # y contient 150 échantillons, 3 classes

### Etude des données

Avant d'utiliser un algorithme d'apprentissage, il faut étudier les données afin de connaître ce qui va être manipulé: leur type, leur plage de valeur, leur répartition par classe. Mais aussi, ceci peut permettre d'avoir une intuition des relations potentielles entre certaines caractéristiques et orienté ainsi le choix d'un type de méthodes d'apprentissage à mettre en place. 

- Afficher la taille (shape) de X et de Y  et le type des données qu'ils contiennent  
- Afficher les données  
- Représenter graphiquement les données => voir ci-dessous le graphique final qui est demandé (vous pouvez vous aider de ce [tutoriel](http://www.python-simple.com/python-matplotlib/pyplot.php))
  1) réaliser des histogrammes pour visualiser la répartition des données de X pour chaque caractéristique
  2) réaliser un camembert pour visualiser la répartition des différents types d'iris.


<p align="center">
<img src="images/iris1.png" />
</p>

In [ ]:
print(X.shape)
print(Y.shape)
print(X.dtype)
print(Y.dtype)
print(X[:])
print(Y[:])

In [ ]:
# création d'une boucle for pour afficher tous les graphiques
# nombre de colonne
n = X.shape[1]
labelAxis=["longueur sepale", "largeur sepale", "longueur petale", "largeur petale" ]
plt.figure(figsize=(12, 14))
for i in range(n):
    plt.subplot(3, 3, i+1)
    plt.hist(X[:, i])
    plt.ylabel("nombre")
    plt.xlabel(labelAxis[i])
    

plt.suptitle("Représentation du dataset IRIS")
plt.tight_layout()

targetCounts=np.bincount(Y)
plt.subplot(3, 3, 5)
plt.pie(targetCounts, labels=['Setosa','Versicolor','Virginica'], autopct = lambda x: str(round(x, 2)) + '%')
plt.title("Répartition des iris")
#plt.savefig("images/exo2Fig2.png")

> Que constatez vous à la lecture de ces graphiques ?  

> caractéristiques pas toute dans la même plage et les données sont uniformément réparties par rapport aux classes

Nous allons maintenant étudier plus en détail la caractéristique *longueur de sépale*. Reproduisez la représentation graphique suivante:

<p align="center">
<img src="images/iris2.png" />
</p>




> Que constatez vous à la lecture de ces graphiques ?  
on pourrait n'utiliser que 2 caractéristiques

In [ ]:
# création d'une boucle for pour afficher tous les graphiques
# nombre de colonne
n = X.shape[1]
labelAxis=["longueur sepale", "largeur sepale", "longueur petale", "largeur petale" ]
plt.figure(figsize=(12, 8))
for i in range(1,n):
    plt.subplot(n//2, n//2, i)
    # on affiche la colonne 0 en fonction des autres colonnes, matplotlib génère 3 couleurs
    plt.scatter(X[:, 0], X[:, i], c=Y) # affiche la variable i en fonction de la variable 0
    plt.xlabel(labelAxis[0])
    plt.ylabel(labelAxis[i])
    plt.colorbar(ticks=list(np.unique(Y)))

plt.suptitle("Représentation du dataset IRIS")

### Prétraitement des données

Nous avons vu précédemment que les valeurs associées aux données ne sont pas dans le même intervalle de valeurs, nous allons donc maintenant les standardiser. Ce traitement est important car il permet de mettre les variables à la même échelle. Les algorithmes d'apprentissage automatique fonctionnent ainsi de manière plus efficace et donnent de meilleurs résultats. Sans standardisation, des variables avec des plages de valeurs très différentes peuvent biaiser les modèles en privilégiant les variables avec des valeurs plus élevées.  
- Réaliser la standardisation en utilisant la classe *StandardScaler* et sa méthode *fit_transform(...)*
- Réafficher le premier graphique pour visualiser les effets du traitement.

In [ ]:
from sklearn.preprocessing import StandardScaler
#TODO
print(X.shape)
print(X[0:10,:])
print(Y.shape)
print(Y[0:150])

scaler = StandardScaler()
X_standard = scaler.fit_transform(X)


In [ ]:
# création d'une boucle for pour afficher tous les graphiques
# nombre de colonne
X=X_standard
n = X.shape[1]
labelAxis=["longueur sepale", "largeur sepale", "longueur petale", "largeur petale" ]
plt.figure(figsize=(12, 14))
for i in range(n):
    plt.subplot(3, 3, i+1)
    plt.hist(X[:, i])
    plt.ylabel("nombre")
    plt.xlabel(labelAxis[i])
    

plt.suptitle("Représentation du dataset IRIS")
plt.tight_layout()

targetCounts=np.bincount(Y)
plt.subplot(3, 3, 5)
plt.pie(targetCounts, labels=['Setosa','Versicolor','Virginica'], autopct = lambda x: str(round(x, 2)) + '%')
plt.title("Répartition des iris")

### Prédiction des classes de l'ensemble de test

- Commencez par séparer le jeu de données en un ensemble d'apprentissage et en un ensemble de test (80% apprentissage, 20% test). Vous travaillez bien sûr avec les données standardisées.

- Réaliser la classification avec *k=3*, calculer l'exactitude de la prédiction et afficher la matrice de confusion.

- Donner la micro-moyenne et la macro-moyenne par rapport à la précision.


In [ ]:

#TODO

X_train, X_test, Y_train, Y_test = train_test_split(X_standard, Y, test_size = 0.2, random_state = 0)
classifier = KNeighborsClassifier(n_neighbors=3)
# Fitting the model
classifier.fit(X_train, Y_train)

# Predicting the Test set results
prediction = classifier.predict(X_test)

print(accuracy_score(Y_test, prediction))
cm = confusion_matrix(Y_test, prediction)
print(cm)
print(precision_score(Y_test,prediction, average='micro'))
print(precision_score(Y_test,prediction, average='macro'))







### Utilisation de la validation croisée

#### Détermination de l'exactitude en fonction de différentes valeurs de *k*

Nous avons utilisé une valeur de *k* arbitraire comme paramètre du classifieur. 
Nous allons maintenant utiliser de la validation croisée pour trouver la meilleure valeur de *k* pour nos données (le nombre de données est restreinte) 
- Nous allons tester différentes valeurs de k (1..20). 
- Pour chaque valeur de *k*, nous allons trouver le score moyen obtenu (exactitude) en réalisant une validation croisée à 10 plis (ou folds).
- Le meilleur *k* obtenu sera celui qui maximise l'exactitude calculée.

Mettez en place ce traitement.
- Réaliser une représentation graphique des valeurs obtenues pour l'exactitude en fonction des différentes valeurs de *k*
- Afficher cette valeur. 



In [19]:
from sklearn.model_selection import cross_val_score

# la liste des scores d'exactitude
exactitude_scores = []

# on réalise  une 10 fold cross validation
liste_k=range(1,20)
for k in liste_k:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, Y_train, cv=10, scoring='accuracy')
    exactitude_scores.append(scores.mean())

In [ ]:


plt.figure()
plt.figure(figsize=(15,10))
plt.title('nombre optimal de voisins', fontsize=20, fontweight='bold')
plt.xlabel('Nombre de voisins', fontsize=15)
plt.ylabel('Exactitude', fontsize=15)
plt.plot(liste_k, exactitude_scores)

plt.show()
# finding best k
best_k = liste_k[exactitude_scores.index(max(exactitude_scores))]
print("The optimal number of neighbors is %d." % best_k)

Tester la valeur optimale de *k* que vous avez déterminée précèdemment.
 


In [ ]:
classifier = KNeighborsClassifier(n_neighbors=6)
# Fitting the model
classifier.fit(X_train, Y_train)

# Predicting the Test set results
prediction = classifier.predict(X_test)

print(accuracy_score(Y_test, prediction))

Vous avez vu à l'aide de la seconde représentation graphique demandée que les données semblent être relativement bien séparées en utilisant seulement 2 caractéristiques. Recommencer la classification en n'utilisant que ces 2 caractéristiques du jeu de données. Que constatez vous ?

In [ ]:
#TODO
X_train, X_test, Y_train, Y_test = train_test_split(X_standard, Y, test_size = 0.2, random_state = 0)
X_train=np.delete(X_train,[1,2],axis=1)
X_test=np.delete(X_test,[1,2],axis=1)
classifier = KNeighborsClassifier(n_neighbors=3)
# Fitting the model
classifier.fit(X_train, Y_train)

# Predicting the Test set results
prediction = classifier.predict(X_test)

print(accuracy_score(Y_test, prediction))
cm = confusion_matrix(Y_test, prediction)
print(cm)
print(precision_score(Y_test,prediction, average='micro'))
print(precision_score(Y_test,prediction, average='macro'))


## Pour aller plus loin

- étude de l'influence de paramètres autre que *k* de *KneighborsClassifier*

- Réaliser des classifications sur d'autres jeux de données qui ont déjà été vus (MNIST, Titanic)
